# **01_data_ingestion**

## 📌 Overview
This notebook represents the **first step of the data ingestion process**.

## 🎯 Objective
- Create a catalog and schema named **retailsales**
- Ingest raw data in **CSV format**
- Store the ingested data for further processing
- Maintain version control by connecting the notebook to a **GitHub repository**

## 🧠 Key Concepts Covered
- Databricks Catalog & Schema
- Raw data ingestion
- CSV file handling using Spark
- GitHub integration in Databricks

## 🔄 Ingestion Flow
1. Create the catalog and schema (**retailsales**)
2. Read raw CSV files using Spark
3. Load data into Databricks tables
4. Track changes using GitHub version control


In [0]:
from pyspark.sql import *
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
retail_data = spark.sql("select * from firstcatloag2026.retailsales.shopping_behavior")
retail_data.display()

# 02_bronze_layer

## 📌 Overview
This notebook focuses on the **Bronze layer**, which stores raw data ingested from the source
with minimal transformations. The goal of this layer is to **capture data exactly as received**
while performing basic data profiling checks.

## 🎯 Objectives
- Validate incoming raw data
- Identify **null values** in critical columns
- Detect **duplicate records**
- Preserve data for audit and reprocessing purposes

## 🔍 Data Quality Checks Performed

### 1️⃣ Null Value Check
- Identify columns containing null or missing values
- Understand data completeness issues
- No data is removed or corrected at this stage

### 2️⃣ Duplicate Records Check
- Detect duplicate rows based on business keys
- Analyze data duplication patterns
- Retain duplicates for traceability

## 🔄 Processing Logic
1. Load raw data from the ingestion layer
2. Perform null value analysis on important fields
3. Check for duplicate records
4. Store the validated raw data in the Bronze layer without modification

## ⚠️ Important Note
The Bronze layer **does not clean or transform data**.  
All issues found are logged or analyzed, but the data remains unchanged.



In [0]:
retail_data_null = retail_data.select(
    *[sum(col(c).isNull().cast("int")).alias(c) for c in retail_data.columns]
)

In [0]:
retail_data_null.display()

# **03_silver_layer**

In [0]:
retail_data = spark.sql("select * from firstcatloag2026.retailsales.shopping_behavior")

In [0]:
retail_data.display()

> - **Checking the count of between the ages**

In [0]:
check_between_18_25=retail_data.filter((col("Age")>18) & (col("Age")<25)).groupBy("Gender").count()
check_between_18_25.display()
check_between_25_35=retail_data.filter((col("Age")>25) & (col("Age")<35)).groupBy("Gender").count()
check_between_25_35.display()
check_between_35_45=retail_data.filter((col("Age")>35) & (col("Age")<45)).groupBy("Gender").count()
check_between_35_45.display()

> **Checking which category is getting more based on the age.**

- **Checking category from 18-25 age**

In [0]:
retail_data_category_18_25 = retail_data.filter((col("Age")>18) & (col("Age")<25)).groupBy("Gender","Category","Item Purchased","Discount Applied","Previous Purchases","Location","Season",).count()
retail_data_category_18_25.display()


- **Checking category from 25-35 age**

In [0]:
retail_data_category_25_35 = retail_data.filter((col("Age")>25) & (col("Age")<35)).groupBy("Gender","Category","Item Purchased","Discount Applied","Previous Purchases","Location","Season",).count()
retail_data_category_25_35.display()



- **Checking category from 35-45 age**

In [0]:
retail_data_category_35_45 = retail_data.filter((col("Age")>35) & (col("Age")<45)).groupBy("Gender","Category","Item Purchased","Discount Applied","Previous Purchases","Location","Season",).count()
retail_data_category_35_45.display()


- **Checking category from 45-55 age**

In [0]:
retail_data_category_45_55 = retail_data.filter((col("Age")>45) & (col("Age")<55)).groupBy("Gender","Category","Item Purchased","Discount Applied","Previous Purchases","Location","Season",).count()
retail_data_category_45_55.display()

- Age-based segmentation
- Derived columns for analytics

In [0]:
retail_data_ages = retail_data.withColumn(
    "Age_Group",
    when((col("Age") > 18) & (col("Age") < 25), "18-25")
    .when((col("Age") >= 25) & (col("Age") < 35), "25-35")
    .when((col("Age") >= 35) & (col("Age") < 45), "35-45")
    .when((col("Age") >= 45) & (col("Age") < 55), "45-55")
    .otherwise("55+")
)

In [0]:
retail_analysis_age = (
    retail_data_ages.select(
        "Age_Group",
        "Gender",
        "Category",
        "Item Purchased",
        "Discount Applied",
        "Previous Purchases",
        "Location",
        "Season"
    )
    .groupBy("Age_Group", "Gender", "Category", "Item Purchased", "Discount Applied", "Previous Purchases", "Location", "Season")
    .count()
)

In [0]:
retail_analysis_age.display()

In [0]:


retail_silver = (
    retail_data
    .withColumn(
        "Discounted",
        when(col("Discount Applied") == "Yes", 1).otherwise(0)
    )
    .drop("Discount Applied")
    .withColumn(
        "high_value_customer",
        when(col("Previous Purchases") >= 10, 1).otherwise(0)
    )
    .withColumnRenamed("Customer ID","Customer_ID")
    .withColumnRenamed("Item Purchased","Item_Purchased")
    .withColumnRenamed("Previous Purchases", "Previous_Purchases")
    .withColumnRenamed("Purchase Amount (USD)", "Purchase_Amount")
    .withColumnRenamed("high_value_customer", "High_Value_Customer")
    .withColumnRenamed("Review Rating","Review_Rating")
    .withColumnRenamed("Subscription Status","Subscription_Status")
    .withColumnRenamed("Shipping Type","Shipping_Type")
    .withColumnRenamed("Promo Code Used","Promo_Code_Used")
    .withColumnRenamed("Payment Method","Payment_Method")
    .withColumnRenamed("Frequency of Purchases","Frequency_of_Purchases")
    
)

retail_silver.show()


In [0]:
%python
retail_analysis_age_clean = retail_analysis_age.toDF(
    *[col.replace(" ", "_").replace(",", "").replace(";", "").replace("{", "")
      .replace("}", "").replace("(", "").replace(")", "").replace("\n", "")
      .replace("\t", "").replace("=", "") for col in retail_analysis_age.columns]
)

retail_analysis_age_clean.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("firstcatloag2026.retailsales.retail_analysis_age")

In [0]:
retail_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .saveAsTable("firstcatloag2026.retailsales.retail_silver")


In [0]:
retail_analysis = spark.sql("select * from firstcatloag2026.retailsales.retail_analysis_age")
retail_analysis.display()

In [0]:
age_category_demand = (
    retail_analysis
    .groupBy("Age_Group", "Category")
    .count()
    .orderBy("age_group", desc("count"))
)
age_category_demand.display()


In [0]:
data_quality_metrics = retail_silver.select(
    count("*").alias("total_records"),
    sum(col("Discounted")).alias("discounted_orders"),
    sum(col("high_value_customer")).alias("high_value_customers")
)

data_quality_metrics.display()


In [0]:
data_quality_metrics.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("firstcatloag2026.retailsales.data_quality_metrics")
